# The abond command

Package pyndynpd is able to estimate dynamic panel models that take a form as follows:

```
df = pd.read_csv("test_data.csv")
command_str='y L(1:?).y L(1:?).x  | gmm(y, 2:3) iv(L(1:1).x)| timedumm'
mydpd = regression.abond(command_str, df, ['id', 'year'])
```
With lag (2 2) I have instructed Stata to use only the second lag of the endogenous variables asinstruments. Due to the small number of countries in my sample a large number of instrumentscauses the Sargan test (explained below) to be weak. The rule of thumb is to keep the number ofinstruments less than or equal to the number of groups. Stata warns you about that at the top of theoutput table. The second lag is required, because it is not correlated with the current error term,while the first lag is. Generally, one can experiment with a second or deeper lags to find a goodinstrument, but using deeper lags reduces sample size. If the number of countries is large enough,one may use all available lags (second and deeper lags) as instruments.

The second list of explanatory variables, iv ( ) (or ivstyle ( )), lists all strictly exogenous variables(l.growth, uncert, tot, dev_m2) as well as the additional instrumental variables (fin_integr,trans_index, flows_eeca), which are not part of equation (1) and, therefore, are not listed beforethe comma in the Stata command. What this option essentially does for the included exogenousvariables is tell Stata to use the variables themselves as their own instruments.

nolevel (or noleveleq) tells Stata to apply the difference GMM estimator. By default xtabond2 willapply the system GMM, if you don’t specify nolevel. (System GMM is discussed next.)
twostep specifies that the two-step estimator is calculated instead of the default one-step.



$$y_{it}=\sum_{j=1}^{p}\alpha_{j}y_{i,t-j}+\sum_{k=1}^{m}\sum_{j=0}^{q_{k}}\beta_{jk}r_{i,t-j}^{(k)}+\boldsymbol{\delta}\boldsymbol{d_{i,t}}+\boldsymbol{\gamma}\boldsymbol{s_{i,t}}+u_{i}+\epsilon_{it} $$ 

In the model above, $y_{i,t-j}$ ($j=1,2,\ldots,p$) denotes a group of $p$ lagged dependent variables. $r_{i,t-j}^{(k)}$ represents a group of $m$ endogeneous variables other than lagged $y$. $\boldsymbol{d_{it}}$ is a vector of predetermined variables which may potentially correlate with past errors, $\boldsymbol{s_{it}}$ is a vector of exogenous variables, and $u_{i}$ represents fixed effect. For illustration purpose, let's consider a basic form of dynamic panel model:

$$
\begin{align}
y_{it}=\alpha_{1}y_{i,t-1}+\delta d_{i,t}+u_{i}+\epsilon_{it} \label{basic}\tag{1}
\end{align}
$$

As lagged dependent variable $y_{i,t-1}$ is included as regressor, the popular techniques in static panel models, such as fixed-effect and first-difference estimators, no longer produce consistent results. Researchers have developed many methods to estimate dynamic panel model. Essentially there are two types of GMM estimates, difference GMM and system GMM.

## Difference GMM 

Difference GMM was developed by [@arellano1991some]. The first step in the process is to eliminate the fixed-effect term $u_{i}$. First differencing Eq ($\ref{basic}$) yields:

$$
\begin{align}
\Delta y_{it}=\alpha_{1}\Delta y_{i,t-1}+\delta\Delta d_{i,t}+\Delta\epsilon_{it}\label{fd}\tag{2} 
\end{align}$$

In the model above, $\Delta y_{i,t-1}$ correlates with $\Delta\epsilon_{i,t}$ because $\Delta y_{i,t-1}=y_{i,t-1}-y_{i,t-2}$, $\Delta\epsilon_{i,t}=\epsilon_{i,t}-\epsilon_{i,t-1}$, and $y_{i,t-1}$ is affected by $\epsilon_{i,t-1}$. As a result, estimating Eq ($\ref{fd}$) directly produces inconsistent result. Instrumental variables are used to solve the issue. [@arellano1991some]suggest to use all lagged $y$ dated $t-2$ and earlier (i.e., $y_{i,1}$, $y_{i,2}$,\..., $y_{i,t-2}$) as instruments for $\Delta y_{i,t-1}$. Similarly, the instruments for predetermined variable $\Delta d_{it}$ include $d_{i,1}$, $d_{i,2}$,\..., $d_{i,t-1}$. Let $z_{i}$ be the instrument variable matrix for individual i:

$$
z_{i}=\left[\begin{array}{ccccccccccccccccccc}
y_{i1} & 0 & 0 & \ldots & \ldots & 0 & \ldots & 0 & d_{i1} & d_{i2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \ldots & 0\\
0 & y_{i1} & y_{i2} & 0 & 0 & 0 & \ldots & 0 & 0 & 0 & d_{i1} & d_{i2} & d_{i3} & 0 & 0 & 0 & 0 & \ldots & 0\\
\vdots & 0 & 0 & \vdots &  &  & \ldots &  &  &  &  &  &  & \vdots &  &  &  & \ldots & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \ldots & y_{i,T-2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \ldots & d_{i,T-1}
\end{array}\right]$$

Difference GMM is based on the moment condition $E(z_{i}^{\prime}\Delta\epsilon_{i})=0$ where $\Delta \epsilon_{i}=(\Delta \epsilon_{i2}, \Delta\epsilon_{i3}\textrm{, }...,\Delta\epsilon_{iT})^{\prime}$ and $z_{i}$ is the instrument variable matrix. Applying this moment condition to sample data, we have $(1/N)\sum_{i=1}^{N}z{}_{i}^{\prime}(\Delta y_{i}-\theta\Delta x_{i})=0$ where $\theta=(\alpha_{1},\delta)'$ and $\Delta x_{i}=(\Delta y_{i,t-1},\Delta d_{it})$ for t=3, \... T. When the number of instruments is greater than the number of independent variables, the moment condition is overidentified and in general there is no $\theta$ available to satisfy the moment condition. Instead, we look for a $\theta$ to minimize moment condition. That is:

$$\hat{\theta}_{gmm}=\arg\min_{\theta}\left(\frac{1}{N}\sum_{i=1}^{N}(\Delta y_{i}-\theta\Delta x_{i})^{\prime}z_{i}\right)W\left(\frac{1}{N}\sum_{i=1}^{N}z^{\prime}{}_{i}(\Delta y_{i}-\theta\Delta x_{i})\right)$$

where W is the weighting matrix of the moments. There are two popularly used weighting matrixes. In a one-step GMM estimate, the weighting matrix is

$$W_{1}=\left(\frac{1}{N}Z^{\prime}H_{1}Z\right)^{-1}$$

where matrix H has twos in the main diagnols, minus ones in the first subdiagnols, and zeros elsewhere:

$$H_{1}=\left[\begin{array}{cccccc}
2 & -1 & 0 & 0 & \ldots & 0\\
-1 & 2 & -1 & 0 & \ldots & 0\\
0 & \ddots & \ddots & \ddots & \ddots & \vdots\\
\vdots & \ddots & -1 & 2 & -1 & 0\\
0 & \ddots & 0 & -1 & 2 & -1\\
0 & \ldots & 0 & 0 & -1 & 2
\end{array}\right]$$

On the other hand, in a two-step GMM estimate, the weighting matrix is

$$W_{2}=\left(\frac{1}{N}Z^{\prime}H_{2}Z\right)^{-1}$$

where $H_{2}=\Delta\hat{\epsilon}\Delta\hat{\epsilon}^{\prime}$ and $\Delta\hat{\epsilon}$ is the residual from one-step GMM.

## System GMM

Compared with difference GMM, sytem GMM adds additional moment conditions, resulting in more instruments:

$$
\begin{align}
z_{i}=\left[\begin{array}{cccccccccccccccc|cccccccc}
y_{i1} & 0 & 0 & 0 & 0 & 0 & \ldots & 0 & d_{i1} & d_{i2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \ldots & 0 & 0 & 0 & 0 & \ldots & 0\\
0 & y_{i1} & y_{i2} & 0 & 0 & 0 & \ldots & 0 & 0 & 0 & d_{i1} & d_{i2} & d_{i3} & 0 & 0 & 0 & 0 & \ldots & 0 & 0 & 0 & 0 & \ldots & 0\\
 &  &  & \vdots &  &  & \ldots &  & \vdots &  &  &  &  & \vdots & \ldots & \vdots &  & \ldots & 0 &  &  &  & \ddots & 0\\
0 & 0 & 0 & 0 & 0 & 0 & \ldots & y_{i,T-2} & 0 & 0 & 0 & 0 & 0 & 0 & \ldots & d_{i,T-1} & 0 & \ldots & 0 & 0 & 0 & 0 & 0 & 0\\
\hline 0 & \ldots & 0 &  &  &  &  &  &  &  &  &  &  &  & 0 & 0 & \Delta y_{i2} & \ldots & 0 & 0 & \Delta d_{i3} &  & 0\\
\vdots &  &  &  &  &  &  &  &  &  &  &  &  &  &  & 0 & 0 & \Delta y_{i3} & \ldots & 0 & 0 & \Delta d_{i4} &  & 0\\
\vdots &  &  &  &  &  &  &  &  &  &  &  &  &  &  & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots &  & \ddots\\
0 & \ldots &  &  &  &  &  &  &  &  &  &  &  &  & \ldots & 0 & 0 & 0 & \ldots & \Delta y_{i,T-1} & 0 & 0 & \ldots & \Delta y_{i,T}
\end{array}\right] \label{z_sys}\tag{3}
\end{align}$$

$$\hat{\theta}_{gmm}=\arg\min_{\theta}\left(\frac{1}{N}\sum_{i=1}^{N}(\widetilde{y}_{i}-\theta\widetilde{x_{i}})^{\prime}z_{i}\right)W\left(\frac{1}{N}\sum_{i=1}^{N}z^{\prime}{}_{i}(\widetilde{y}-\theta\widetilde{x_{i}})\right)$$

where $$\widetilde{y}=\left(\begin{array}{c}
\Delta y_{i}\\
\hline y_{i}
\end{array}\right)\textrm{ and }\widetilde{x_{i}}=\left(\begin{array}{c|c}
\Delta x_{i} & 0\\
\hline x_{i} & 1
\end{array}\right)$$

## Robust estimation of coefficients' covariance 

## Specification Test 

### Error serial correlation test 

Second-order serial correlation test: if $\epsilon_{it}$ in Eq ($\ref{basic}$) is serially correlated, GMM estimates are no longer consistent. In a first-differenced model （e.g., Eq ($\ref{fd}$)), to test whether $\epsilon_{i,t-1}$ is correlated with $\epsilon_{i,t-2}$, the second-order autocovariance of the residuals, $\textrm{AR(2)}$, is calculated as:

$$AR(2)=\frac{b_{0}}{\sqrt{b_{1}+b_{2}+b_{3}}}\textrm{ where}$$

$$b_{0}=\sum_{i=1}^{N}\Delta\hat{\hat{\epsilon}}_{i}^{\prime}L_{\Delta\hat{\hat{\epsilon}}}^{2}$$

$$b_{1}=\sum_{i=1}^{N}L_{\Delta\hat{\hat{\epsilon}}_{i}^{\prime}}^{2}H_{2}L_{\Delta\hat{\hat{\epsilon}}_{i}}^{2}$$ 

$$b_{2}=\textrm{-}2\left(\sum_{i=1}^{N}L_{\Delta\hat{\hat{\epsilon}}_{i}^{\prime}}^{2}x_{i}\right)\left[\left(\sum_{i=1}^{N}x_{i}^{\prime}z_{i}\right)W_{2}\left(\sum_{i=1}^{N}z_{i}^{\prime}x_{i}\right)\right]^{-1}\left(\sum_{i=1}^{N}x_{i}^{\prime}z_{i}\right)W_{2}\left(\sum_{i=1}^{N}z_{i}^{\prime}H_{2}L_{\Delta\hat{\hat{\epsilon}}_{i}}^{2}\right)$$

$$b_{3}=\left(\sum_{i=1}^{N}L_{\Delta\hat{\hat{\epsilon}}_{i}^{\prime}}^{2}x_{i}\right)\hat{V}_{\hat{\hat{\theta}}}\left(\sum_{i=1}^{N}x_{i}^{\prime}L_{\Delta\hat{\hat{\epsilon}}_{i}}^{2}\right)$$

### Hansen overidentification test 

Hansen overidentification test is used to check if instruments are exogeneous. Under the null hypothesis that instruments are valid, test statistic, $S$, should be close to zero:

$$S=\left(\sum_{i=1}^{N}\Delta\hat{\hat{\epsilon}}_{i}^{\prime}z_{i}\right)W_{2}\left(\sum_{i=1}^{N}z_{i}^{\prime}\Delta\hat{\hat{\epsilon}}_{i}\right)$$

# Handling instrument proliferation issue 

Difference GMM and system GMM may generate too many instruments, which causes several problems (citation). Package pydynpd allows users to reduce the number of instruments in two ways. First, users can control the number of instruments in command string. For example, $\textrm{gmm(w, 2:3)}$ states that only $n_{t-2}$ and $n_{t-3}$ are used as instruments, rather than all lagged $n$ dated $t-2$ and earlier. Second, users can choose to collapse the instrumental variable matrix. For example, if collapsed, matrix as in Eq ($\ref{z_sys}$) is changed to:

$$z_{i}=\left[\begin{array}{cccccccccc|cc}
y_{i1} & 0 & 0 & \ldots & 0 & d_{i1} & d_{i2} & 0 & \ldots & 0\\
y_{i1} & y_{i2} & 0 & \ldots & 0 & d_{i1} & d_{i2} & d_{i3} & \ldots & 0\\
\vdots & \vdots & \ddots & \ldots & \vdots &  &  &  & \ddots & \vdots\\
y_{i1} & y_{i2} & y_{i3} & \ldots & y_{i,T-2} & d_{i1} & d_{i2} & d_{id} & \ldots & d_{i,T-1}\\
\hline 0 &  &  & \ldots & 0 & 0 & 0 & 0 & \ldots & 0 & \Delta y_{i2} & \Delta d_{i3}\\
0 & 0 &  &  & 0 & 0 & 0 & 0 & \ldots & 0 & \Delta y_{i3} & \Delta d_{i4}\\
\vdots &  & \ddots &  & \vdots &  &  & \vdots &  & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \ldots & 0 & 0 &  & 0 & \ldots & 0 & \Delta y_{i,T-1} & \Delta d_{iT}
\end{array}\right]$$

This change dramatically reduces the number of instruments. Intuitively, the number of instruments is positively associated with the width of the matrix above.

